In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch

# ----------------- CONFIG -----------------
lora_model_path = "./legal-mistral-lora"

# Load config to get base model
config = PeftConfig.from_pretrained(lora_model_path)
base_model_id = config.base_model_name_or_path

# ----------------- Load tokenizer -----------------
tokenizer = AutoTokenizer.from_pretrained(lora_model_path)

# ----------------- Load base model -----------------
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# ----------------- Load fine-tuned LoRA weights -----------------
model = PeftModel.from_pretrained(base_model, lora_model_path)
model.eval()

# ----------------- Generate Response -----------------
def generate(prompt, max_new_tokens=300):
    input_text = f"<s>[INST] {prompt.strip()} [/INST]"
    inputs = tokenizer(input_text, return_tensors="pt")
    inputs = {k: v.to("mps") for k, v in inputs.items()}  

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ----------------- Test -----------------
response = generate( """
JURY DEMAND REQUESTED

I. PARTIES

1. Plaintiff Tech Solutions, Inc. is a technology manufacturing company with its principal place of business in New York, New York.

2. Defendant Global Supplies LLC is a wholesale supplier of electronic components, headquartered in Dallas, Texas, and engaged in interstate commerce.

II. JURISDICTION AND VENUE

1. This Court has jurisdiction pursuant to 28 U.S.C. § 1332, as the amount in controversy exceeds $150,000, and the parties are citizens of different states.

2. Venue is proper under 28 U.S.C. § 1391, as Defendant conducted business in this district, and the contract in dispute was executed within New York County, New York.

III. FACTUAL ALLEGATIONS

1. On or about February 5, 2025, Plaintiff entered into a written contract with Defendant for the purchase of 5,000 microprocessor chips to be used in the production of Plaintiff’s new product line.

2. The terms of the agreement, governed by Article 2 of the Uniform Commercial Code (UCC) – Sale of Goods, required Defendant to deliver the microprocessors no later than April 1, 2025, at a total cost of $250,000, payable upon receipt.

3. Defendant failed to deliver the microprocessors on the agreed date, constituting a material breach of contract under UCC § 2-601.

4. Plaintiff issued a written demand for performance on April 5, 2025, providing Defendant an opportunity to cure the breach pursuant to UCC § 2-508, but Defendant failed to act.

5. Due to Defendant’s breach, Plaintiff was forced to secure an alternative supplier at a higher cost, incurring additional expenses of $75,000 and experiencing production delays that resulted in lost revenue.


Choose only one category exactly as written from the list below. Do not add any text or explanation:

Other Matters - Consumer Credit (Card) Original Creditor Plaintiff

Other Matters - Contract - Other

Torts - Other

Other Matters - Consumer Credit (Card) Secondary Debt Buyer Plaintiff

Commercial - Contract

Torts - Other Negligence

Other Matters - Consumer Credit (Non-Card) Transaction

Torts - Motor Vehicle

Other Matters - Consumer Credit (Card) Debt Buyer Plaintiff

Torts - Asbestos

Real Property - Other

Real Property - Mortgage Foreclosure - Residential

Commercial - Business Entity

Special Proceedings - CPLR Article 78

Commercial - Other

Real Property - Mortgage Foreclosure - Commercial

Other Matters - Local Court Appeal

Real Property - Partition

Other Matters - Contract Non-Commercial

Torts - Medical, Dental, or Podiatrist Malpractice

Special Proceedings - Other

Commercial - Insurance

Gender-Motivated Violence Act (GMVA)

Torts - Other Professional Malpractice

Real Property - Tax Foreclosure

Special Proceedings - CPLR Article 75

Commercial - UCC

Other Matters - Mechanic's Lien

Torts - Environmental

Commercial - Contract - Commercial Division

Torts - Product Liability

Other Matters - Civil Forfeiture

Other Real Property - Unsafe Buildings

Commercial Division

Commercial Division - Business Entity

Commercial - Other - Commercial Division

Special Proceedings - CPLR Article 75 - Uninsured Motorist

Other Real Property - Foreclosure on Mechanic's Lien

Special Proceeding - CPLR Article 78 - DHCR

Commercial - Business Entity - Commercial Division

Commercial Division - Contract

Real Property - Condemnation

Other Matters - Agriculture/Market

Covid - 19 Action Against Nursing Home

Other Real Property - Non-Mortgage Foreclosure

Commercial Division - Other

Special Proceedings - Extreme Risk Protection Order

Special Proceedings - Mechanic's Lien


""")
print("\n📜 Generated Response:\n", response)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


📜 Generated Response:
 [INST] JURY DEMAND REQUESTED

I. PARTIES

1. Plaintiff Tech Solutions, Inc. is a technology manufacturing company with its principal place of business in New York, New York.

2. Defendant Global Supplies LLC is a wholesale supplier of electronic components, headquartered in Dallas, Texas, and engaged in interstate commerce.

II. JURISDICTION AND VENUE

1. This Court has jurisdiction pursuant to 28 U.S.C. § 1332, as the amount in controversy exceeds $150,000, and the parties are citizens of different states.

2. Venue is proper under 28 U.S.C. § 1391, as Defendant conducted business in this district, and the contract in dispute was executed within New York County, New York.

III. FACTUAL ALLEGATIONS

1. On or about February 5, 2025, Plaintiff entered into a written contract with Defendant for the purchase of 5,000 microprocessor chips to be used in the production of Plaintiff’s new product line.

2. The terms of the agreement, governed by Article 2 of the Uniform